<img style="float: center;" src="images/CI_horizontal.png" width="600">
<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Website</a>
    </span>
</center>

Ghani, Rayid, Frauke Kreuter, Julia Lane, Brian Kim, Adrianne Bradford, Alex Engler, Nicolas Guetta Jeanrenaud, Graham Henke, Daniela Hochfellner, Clayton Hunter, Avishek Kumar, Jonathan Morgan, Ekaterina Levitskaya.

# Text Analysis

Text analysis is used to extract useful information from or summarize a large amount of unstructured text stored in documents. This opens up the opportunity of using text data alongside more conventional data sources (e.g. surveys and administrative data). The goal of text analysis is to take a large corpus of complex and unstructured text data and extract important and meaningful messages in a comprehensible way. 

Text analysis can help with the following tasks:

* **Information Retrieval**: Find relevant information in a large database, such as a systematic literature review, that would be very time-consuming for humans to do manually. 

* **Clustering and Text Categorization**: Summarize a large corpus of text by finding the most important phrases, using methods like topic modeling. 

* **Text Summarization**: Create category-sensitive text summaries of a large corpus of text. 

* **Machine Translation**: Translate documents from one language to another. 

In this notebook, we are going to analyze abstracts of grants using topic modeling to examine the content of our data and document classification to tag academic discipline in the abstracts. 

## Learning Outcomes

In this notebook, you will...
* Learn how to transform a corpus of text into a structured matrix format so that we can apply natural language processing (NLP) methods
* Learn the basics and applications of topic modeling
* Learn how to do document tagging and evaluate the results

 
## Glossary of Terms

Glossary of Terms:

* **Corpus**: A corpus is the set of all text documents used in your analysis; for example, your corpus of text may include hundreds of research articles.

* **Tokenize**: Tokenization is the process by which text is separated into meaningful terms or phrases. In English this is easy to do for individual words, as they are separated by whitespace; however, it can get more complicated to  automate determining which groups of words constitute meaningful phrases. 

* **Stemming**: Stemming is normalizing text by reducing all forms or conjugations of a word to the word's most basic form. In English, this can mean making a rule of removing the suffixes "ed" or "ing" from the end of all words, but it gets more complex. For example, "to go" is irregular, so you need to tell the algorithm that "went" and "goes" stem from a common lemma, and should be considered alternate forms of the word "go."

* **TF-IDF**: TF-IDF (term frequency-inverse document frequency) is an example of feature engineering where the most important words are extracted by taking into account their frequency in documents and the entire corpus of documents as a whole.

* **Topic Modeling**: Topic modeling is an unsupervised learning method where groups of words that often appear together are clustered into topics. Typically, the words in one topic should be related and make sense (e.g. boat, ship, captain). Individual documents can fall under one topic or multiple topics. 

* **LDA**: LDA (Latent Dirichlet Allocation) is a type of probabilistic model commonly used for topic modeling. 

* **Stop Words**: Stop words are words that have little semantic meaning but occur very frequently, like prepositions, articles and common nouns. For example, every document (in English) will probably contain the words "and" and "the" many times. You will often remove them as part of preprocessing using a list of stop words.


# Setup

In [1]:
# database connection
from sqlalchemy import create_engine

# data manipulation
import pandas as pd
import numpy as np

# text analysis tools
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn import preprocessing
from nltk import SnowballStemmer
import string

In [2]:
# Database Connection
host = 'stuffed.adrf.info'
DB = 'appliedda'

connection_string = "postgresql://{}/{}".format(host, DB)
conn = create_engine(connection_string)

## Motivation: Grant Proposal Abstracts

One of the pieces of data you have access to in this class is the NIH Federal RePORTER data, containing information about grants. In this notebook, we will use text analysis using the grant proposal abstracts in order to determine what types of topics these abstracts are about. Note that we do not have a pre-compiled list of topics that we are matching to the abstracts. Instead, we want to explore the data and use unsupervised learning in order to generate the topics.

### Load the Data

To start, we'll load the data into a pandas DataFrame from the database. We will be analyzing the abstracts of grant proposals. For demonstration purposes of this class we will only use year 2015 and we will load 2,000 abstracts (in order to be able to run the model faster in class, but you can load the full dataset in your own time, to look at the results). 

In [3]:
# Load data into dataframe
query = '''
SELECT *
FROM ada_ncses_2019.nih_fedreporter_grants
LIMIT 2000
'''
abstracts = pd.read_sql(query,conn)

## Explore the Data
Our text data table has 2 fields:

- `project_id` - project ID.
- `abstract` - text of abstract.

Let's take a look at examples of the values:

In [4]:
abstracts.head()

,project_id,abstract
0,725546,DESCRIPTION (provided by applicant): Tradition...
1,725600,DESCRIPTION (provided by applicant): Fungal in...
2,725601,DESCRIPTION (provided by applicant): The overa...
3,725608,DESCRIPTION (provided by applicant): Death fro...
4,725657,Dopamine is important for the rewarding and re...


We will save the values of the column `abstract` to a list.

In [6]:
abstracts_list = abstracts['abstract'].values.tolist()

In [8]:
#look at first abstract
abstracts_list[0]

'DESCRIPTION (provided by applicant): Traditional protein site-directed mutagenesis has served biochemists for many years but is lacking when one seeks to construct libraries of mutants such as in alanine scanning. In protein engineering experiments, protein libraries are redundant mixtures requiring more screening or selection than would be necessary from a  perfect  mixture. This proposal seeks to create a simple, fast, and effective way to perform scanning codon mutagenesis throughout any protein coding sequence. Coupled with a reading frame selection, this will allow the rapid production of collections of proteins that contain substitutions of a defined set of amino acids, including those that contain unnatural functional groups. The proposed studies will i)develop a Mu transposon-based method for the random insertion and deletion of in-frame codon mutations from an open reading frame, ii) verify this methodology by performing whole-gene alanine scanning on an a counterselectable e

## Topic Modeling
We are going to apply topic modeling, an unsupervised learning method, to our corpus to find the high-level topics in our corpus. Through this process, we'll discuss how to clean and preprocess our data to get the best results. Topic modeling is a broad subfield of machine learning and natural language processing. We are going to focus on a common modeling approach called Latent Dirichlet Allocation (LDA).

To use topic modeling, we first have to assume that topics exist in our corpus, and that some small number of these topics can "explain" the corpus. Topics in this context refer to words from the corpus, in a list that is ranked by probability. A single document can be explained by multiple topics. For instance, an article on net neutrality would fall under the topic "technology" as well as the topic "politics". The set of topics used by a document is known as the document's allocation, hence, the name Latent Dirichlet Allocation, each document has an allocation of latent topics allocated by Dirichlet distribution. Latent (or hidden) stands for topics in the documents that are existing but not yet developed or manifest and which can be discovered based on observed data, such as words in the documents. Dirichlet refers to distributions that are taken into account when creating topics: a distribution of words in the topic (which words are more or less probable to belong to a given topic) and a distribution of topics in documents (which topic is more or less probable for a given document). 

We will use topic modeling in order to determine what types of research is done based on the grants' abstracts.

LDA model takes as input a corpus (a collection of text documents). Every text document is tokenized to become a sequence of words (tokens). All unique words across a given corpus are saved as a vocabulary. Text documents are then converted to a matrix of token counts (how often a given unique word from a vocabulary appears in a given text document), e.g.:

|doc# / unique words | 'science' | 'research' | 'cell' | 'DNA' | 'gene' |
|-----------|--------|---------|-------|-------|------|
|document 1 |    0   |    0    |   1   |   5   |  7   |
|document 2 |    1   |    2    |   0   |   1   |  0   |
|document 3 |    1   |    5    |   2   |   0   |  0   |


The LDA model finds the probability of a word appearing in a given topic, and then maps a probability of a topic being assigned to a given document.

### Preparing Text Data for Natural Language Processing (NLP)

The first important step in working with text data is cleaning and processing the data, which includes (but is not limited to):
- forming a corpus of text
- stemming and lemmatization
- tokenization
- removing stopwords
- finding words co-located together (N-grams)

The ultimate goal is to transform our text data into a form an algorithm can work with, because a document or a corpus of text cannot be fed directly into an algorithm. Algorithms expect numerical feature vectors with certain fixed sizes, and can't handle documents, which are basically sequences of symbols with variable length. We will be transforming our text corpus into a _bag of n-grams_ to be further analyzed. In this form our text data is represented as a matrix where each row refers to a specific job description (document) and each column is the occurence of a word (feature).

### Stemming and Lemmatization - Distilling text data

We want to process our text through _stemming and lemmatization_, or replacing words with their root or simplest form. For example, "systems", "systematic", and "system" are all different words, but we can replace all these words with "system" without sacrificing much meaning. 
- A **lemma** is the original dictionary form of a word (e.g. the lemma for "lies", "lied", and "lying" is "lie").
- The process of turning a word into its simplest form is **stemming**. There are several well-known stemming algorithms -- Porter, Snowball, Lancaster - that all have their respective strengths and weaknesses.

In this notebook, we will use the Snowball Stemmer:

In [9]:
# Examples of how a Stemmer works:
stemmer = SnowballStemmer("english")
print(stemmer.stem('lies'))
print(stemmer.stem('lying'))
print(stemmer.stem('systematic'))
print(stemmer.stem('running'))

lie
lie
systemat
run


### Removing Punctuation

For some purposes, we might want to preserve punctuation. For example, if we wanted to be able to detect sentiment of text, we might want to keep exclamation points, because they signify something about the text. For our purposes, however, we will simply strip the punctuation so that it does not affect our analysis. To do this, we use `string` package, creating a translator that takes any string and "translates" it into a string without any punctuation.

An example using the first abstract in our corpus is shown below.

In [11]:
# Before
abstracts_list[0]

'DESCRIPTION (provided by applicant): Traditional protein site-directed mutagenesis has served biochemists for many years but is lacking when one seeks to construct libraries of mutants such as in alanine scanning. In protein engineering experiments, protein libraries are redundant mixtures requiring more screening or selection than would be necessary from a  perfect  mixture. This proposal seeks to create a simple, fast, and effective way to perform scanning codon mutagenesis throughout any protein coding sequence. Coupled with a reading frame selection, this will allow the rapid production of collections of proteins that contain substitutions of a defined set of amino acids, including those that contain unnatural functional groups. The proposed studies will i)develop a Mu transposon-based method for the random insertion and deletion of in-frame codon mutations from an open reading frame, ii) verify this methodology by performing whole-gene alanine scanning on an a counterselectable e

In [12]:
# Create translator that replaces punctuation with empty spaces
translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))

In [13]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [14]:
# After
abstracts_list[0].translate(translator)

'DESCRIPTION  provided by applicant   Traditional protein site directed mutagenesis has served biochemists for many years but is lacking when one seeks to construct libraries of mutants such as in alanine scanning  In protein engineering experiments  protein libraries are redundant mixtures requiring more screening or selection than would be necessary from a  perfect  mixture  This proposal seeks to create a simple  fast  and effective way to perform scanning codon mutagenesis throughout any protein coding sequence  Coupled with a reading frame selection  this will allow the rapid production of collections of proteins that contain substitutions of a defined set of amino acids  including those that contain unnatural functional groups  The proposed studies will i develop a Mu transposon based method for the random insertion and deletion of in frame codon mutations from an open reading frame  ii  verify this methodology by performing whole gene alanine scanning on an a counterselectable e

### Tokenizing

We want to separate text into individual tokens (generally individual words). To do this, we will first write a function that takes a string and splits it up into individual words.

To tokenize, we will use a `.split()` function.

In [15]:
abstracts_list[0].split()[:10]

['DESCRIPTION',
 '(provided',
 'by',
 'applicant):',
 'Traditional',
 'protein',
 'site-directed',
 'mutagenesis',
 'has',
 'served']

We will do the whole process of removing punctuation, stemming, and tokenizing all in one function.

In [16]:
def tokenize(text):
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))  # translator that replaces punctuation with empty spaces
    return [stemmer.stem(i) for i in text.translate(translator).split()]  # stemmer and tokenizing into words

The `tokenize` function actually does several things at the same time. First, it removes any punctuation using the `translate` method. Then, the `split` method breaks it apart into individual words. Then, using `stemmer.stem`, it creates a list of the stemmed versions of each of those individual words.

Let's take a look at an example of how this works using the first abstract in our corpus.

In [17]:
tokenize(abstracts_list[0])[:20]

['descript',
 'provid',
 'by',
 'applic',
 'tradit',
 'protein',
 'site',
 'direct',
 'mutagenesi',
 'has',
 'serv',
 'biochemist',
 'for',
 'mani',
 'year',
 'but',
 'is',
 'lack',
 'when',
 'one']

What we get out of it is something called a **bag of words**. This is a list of all of the words that are in the abstract, cleaned of all punctuation and stemmed. The paragraph is now represented as as vector of individual words rather than as one whole entity.

We can apply this to each abstract in our corpus using `CountVectorizer`. This will not only do the tokenizing, but it will also count any duplicates of words and create a matrix that contains the frequency of each word. This will be quite a large matrix (number of columns will be number of unique words), so it outputs the data as a sparse matrix.

Similar to how we fit models using `sklearn`, we will first create the `vectorizer` object (you can think of this like a model object), and then fit it with our abstracts. This should give us back our overall corpus bag of words, as well as a list of features (that is, the unique words in all the abstracts).

In [18]:
vectorizer = CountVectorizer(analyzer="word",        # unit of features are single words rather than characters
                            tokenizer=tokenize,      # function to create tokens
                            ngram_range=(0,1),       # unigrams - single words
                            strip_accents='unicode', # remove accent characters
                            min_df = 0.05,           # only include words with minimum frequency of 0.05
                            max_df = 0.95)           # only include words with maximum frequency of 0.95

In [19]:
bag_of_words = vectorizer.fit_transform(abstracts_list)  # transform our corpus as a bag of words
features = vectorizer.get_feature_names()

Let's print some of these objects to get an idea of what they look like. We'll just try printing the first row of the `bag_of_words` matrix as well as the feature names (that is, the column names). Note that because this matrix can get quite large and be quite sparse, the data are stored in a slightly different format, and so the output will look a little different from what you might be used to.

In [20]:
print(bag_of_words[0])

  (0, 171)	1
  (0, 508)	3
  (0, 94)	3
  (0, 52)	1
  (0, 507)	10
  (0, 580)	1
  (0, 182)	1
  (0, 280)	1
  (0, 567)	1
  (0, 251)	4
  (0, 375)	1
  (0, 701)	1
  (0, 93)	1
  (0, 339)	1
  (0, 348)	1
  (0, 688)	1
  (0, 437)	1
  (0, 562)	2
  (0, 410)	1
  (0, 603)	1
  (0, 58)	1
  (0, 216)	2
  (0, 232)	1
  (0, 55)	3
  (0, 538)	1
  :	:
  (0, 669)	2
  (0, 617)	2
  (0, 579)	1
  (0, 64)	1
  (0, 479)	1
  (0, 478)	1
  (0, 408)	1
  (0, 132)	2
  (0, 600)	1
  (0, 45)	1
  (0, 509)	1
  (0, 282)	1
  (0, 532)	1
  (0, 420)	1
  (0, 12)	1
  (0, 625)	1
  (0, 262)	1
  (0, 342)	1
  (0, 643)	1
  (0, 597)	1
  (0, 349)	1
  (0, 386)	1
  (0, 89)	1
  (0, 156)	1
  (0, 65)	1


In [21]:
features[10:30]

['abnorm',
 'about',
 'access',
 'accomplish',
 'achiev',
 'acid',
 'acquir',
 'across',
 'act',
 'action',
 'activ',
 'acut',
 'adapt',
 'addit',
 'address',
 'administr',
 'adult',
 'advanc',
 'affect',
 'after']

Now that we have our bag of words, we can start using for models such as Latent Dirichlet Allocation.

## Latent Dirichlet Allocation

Latent Dirichlet Allocation (LDA) is a statistical model that generates groups based on similarities. This is an example of an **unsupervised machine learning model**. That is, we don't have any sort of outcome variable - we are just trying to group the abstracts into rough categories.

Let's try fitting an LDA model. The way we do it is very similar to the models we have fit before from `sklearn`. We first create a `LatentDirichletAllocation` object, then fit it using our corpus bag of words.

In [22]:
lda = LatentDirichletAllocation(learning_method='online')

doctopic = lda.fit_transform(bag_of_words)

In [23]:
ls_keywords = []
for i,topic in enumerate(lda.components_):
    word_idx = np.argsort(topic)[::-1][:5]
    keywords = ', '.join(features[i] for i in word_idx)
    ls_keywords.append(keywords)
    print(i, keywords)

0 cancer, tumor, dna, therapi, growth
1 that, is, we, will, for
2 that, is, by, will, we
3 infect, hiv, 1, viral, virus
4 brain, will, behavior, neuron, that
5 for, will, be, drug, develop
6 research, for, will, train, program
7 will, for, with, studi, is
8 core, project, alcohol, for, provid
9 cell, t, immun, that, b


Not all of these look very helpful! There are way too many common words in the corpus, such as 'for', 'with', 'that' and so on. We need to remove them, because they don't actually have any interesting information about the documents.

### Removing meaningless text - Stopwords

Stopwords are words that are found commonly throughout a text and carry little semantic meaning. Examples of common stopwords are prepositions ("to", "on", "in"), articles ("the", "an", "a"), conjunctions ("and", "or", "but") and common nouns. For example, the words _the_ and _of_ are totally ubiquitouus, so they won't serve as meaningful features, whether to distinguish documents from each other or to tell what a given document is about. You may also run into words that you want to remove based on where you obtained your corpus of text or what it is about. There are many lists of common stopwords available for you to use, both for general documents and for specific contexts, so you don't have to start from scratch.

We can eliminate stopwords by checking all the words in our corpus against a list of commonly occuring stopwords that comes with NLTK.

> Because ADRF is not connected to the Internet, we downloaded a list of stopwords from NLTK and manually uploaded it to the ADRF. Normally, when you have a connection to the Internet, you can just use `from nltk.corpus import stopwords` and `nltk.download('stopwords')`, to download stopwords directly.

In [24]:
stop = pd.read_csv('stopwords.csv')
stop = stop['stopwords'].values.tolist()
stop[0:15]

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't"]

In [25]:
# Tokenize stop words to match 
eng_stopwords = [tokenize(s)[0] for s in stop]

In [26]:
vectorizer = CountVectorizer(analyzer="word",           # unit of features are single words rather than characters
                            tokenizer=tokenize,         # function to create tokens
                            ngram_range=(0,1),          # unigrams - single words
                            strip_accents='unicode',    # remove accent characters
                            stop_words = eng_stopwords, # remove stopwords
                            min_df = 0.05,              # only include words with minimum frequency of 0.05
                            max_df = 0.95)              # only include words with maximum frequency of 0.95

# Creating bag of words
bag_of_words = vectorizer.fit_transform(abstracts_list) # transform our corpus as a bag of words
features = vectorizer.get_feature_names()               # get features (words)

# Fitting LDA model
lda = LatentDirichletAllocation(n_components = 5, learning_method='online')
doctopic = lda.fit_transform(bag_of_words)

# Displaying the top keywords in each topic
ls_keywords = []
for i,topic in enumerate(lda.components_):
    word_idx = np.argsort(topic)[::-1][:5]
    keywords = ', '.join(features[i] for i in word_idx)
    ls_keywords.append(keywords)
    print(i, keywords)

0 cell, regul, activ, function, mechan
1 effect, use, studi, cancer, increas
2 infect, cell, use, immun, studi
3 provid, core, data, develop, project
4 research, health, studi, develop, provid


### N-grams - Adding context by creating N-grams

Obviously, reducing a document to a bag of words means losing much of its meaning - we put words in certain orders, and group words together in phrases and sentences, precisely to give  them more meaning. If you follow the processing steps we have gone through so far, splitting your document into individual words and then removing stopwords, you will completely lose all phrases like "commander in chief".

One way to address this is to break down each document similarly, but rather than treating each word as an individual unit, treat each group of 2 words, or 3 words, or _n_ words, as a unit. We call is a "bag on _n_-grams", where _n_ is the number of words in each chunk. Then you can analyze which groups of words commonly occur together (in a fixed order).

We will need to change the `ngram_range` variable from `(0,1)` to `(0,2)` to get the  bigrams, to `(0,3)` to get the trigrams, and so on.

In [ ]:
vectorizer = CountVectorizer(analyzer="word",           # unit of features are single words rather than characters
                            tokenizer=tokenize,         # function to create tokens
                            ngram_range=(0,2),          # allow for bigrams
                            strip_accents='unicode',    # remove accent characters
                            stop_words = eng_stopwords, # remove stopwords
                            min_df = 0.05,              # only include words with minimum frequency of 0.05
                            max_df = 0.95)              # only include words with maximum frequency of 0.95

Note that it's entirely possible to not have any bigrams or other n-grams. This just means that there aren't many very common phrases in the corpus.

## TF-IDF - Weighting terms based on frequency

A final step in cleaning and processing our text data is **Term Frequency-Inverse Document Frequency (TF-IDF)**. TF-IDF is based on the idea that the words (or terms) that are most related to a certain topic will occur frequently in documents on that topic, and infrequently in unrelated documents. TF-IDF re-weights words so that we emphasize words that are unique to a document and suppress words that are common throughout the corpus by inversely weighting terms based on their frequency within the document and across the corpus.

We can also try adding to the stopwords list other stopwords that you think might be useful to remove in order to create more meaningful topics:

In [28]:
stop = eng_stopwords + ['provid', 'use', 'studi']
full_stopwords = [tokenize(s)[0] for s in stop]

Let's look at how to use TF-IDF:

In [29]:
vectorizer = CountVectorizer(analyzer="word",            # unit of features are single words rather than characters
                            tokenizer=tokenize,          # function to create tokens
                            ngram_range=(0,2),           # allow for bigrams
                            strip_accents='unicode',     # remove accent characters
                            stop_words = full_stopwords, # remove stopwords
                            min_df = 0.05,               # only include words with minimum frequency of 0.05
                            max_df = 0.95)               # only include words with maximum frequency of 0.95

# Creating bag of words
bag_of_words = vectorizer.fit_transform(abstracts_list)  # transform our corpus as a bag of words
features = vectorizer.get_feature_names()                # get features (words)

# Use TfidfTransformer to re-weight bag of words
transformer = TfidfTransformer(norm = None, smooth_idf = True, sublinear_tf = True)
tfidf = transformer.fit_transform(bag_of_words)

# Fitting LDA model
lda = LatentDirichletAllocation(n_components = 5, learning_method='online')
doctopic = lda.fit_transform(tfidf)

# Displaying the top keywords in each topic
ls_keywords = []
for i,topic in enumerate(lda.components_):
    word_idx = np.argsort(topic)[::-1][:5]
    keywords = ', '.join(features[i] for i in word_idx)
    ls_keywords.append(keywords)
    print(i, keywords)

0 structur, imag, understand, develop, system
1 cell, regul, signal, role, mechan
2 brain, gene, associ, behavior, chang
3 infect, immun, cell, host, viral
4 research, health, train, program, core


In [30]:
ls_keywords

['structur, imag, understand, develop, system',
 'cell, regul, signal, role, mechan',
 'brain, gene, associ, behavior, chang',
 'infect, immun, cell, host, viral',
 'research, health, train, program, core']

In [31]:
# Look at how each abstract (row) fits into each topic (column)
pd.DataFrame(doctopic, columns = ls_keywords)

,"structur, imag, understand, develop, system","cell, regul, signal, role, mechan","brain, gene, associ, behavior, chang","infect, immun, cell, host, viral","research, health, train, program, core"
0,0.856373,0.000767,0.000763,0.000765,0.141331
1,0.291148,0.196006,0.028662,0.483618,0.000565
2,0.053823,0.153774,0.000315,0.223703,0.568385
3,0.770337,0.226821,0.000949,0.000947,0.000947
4,0.089949,0.146318,0.669469,0.000707,0.093556
...,...,...,...,...,...
1995,0.000836,0.494587,0.000836,0.502911,0.000830
1996,0.338457,0.000945,0.587215,0.000952,0.072430
1997,0.000818,0.000823,0.837281,0.000816,0.160261
1998,0.484245,0.304822,0.166557,0.001097,0.043279


<font color=red><h3> Checkpoint: Add other stopwords or change the number of topics</h3></font>

To practice, you can add other stopwords that you think could be useful to remove to get more meaningful topics. You can also play around with changing the number of topics (`n_components` variable).

### Supervised Learning: Document Classification

Topic modeling is an example of an unsupervised learning: we were looking to uncover structure in the form of topics, but we did not necessarily know the ground truth of how many topics there are.

We can also do supervised learning with text data. In supervised learning, we have a _known_ outcome or label (_Y_) that we want to produce given some data (_X_), and in general, we want to be able to produce this _Y_ when we _don't_ know it, or when we _only_ have _X_.

In order to produce labels we need to first have examples our algorithm can learn from, a "training set". In the context of text analysis, developing a training set can be very expensive, as it can require a large amount of human labor or linguistic expertise. **Document classification** is an example of supervised learning in which we want to characterize our documents based on their contents (_X_). A common example of document classification is spam e-mail detection. Another example of supervised learning in text analysis is _sentimenet analysis_, where _X_ is our documents and _Y_ is the state of the author. This "state" is dependent on the question you are trying to answer, and can range from the author being happy or unhappy with a product to the author being politically conservative or liberal. Another example is _part-of-speech tagging_ where _X_ are individual words and _Y_ is the part-of-speech.

In the context of our class, one useful application is, for example, classifying whether a text belongs to a given field of study (e.g. broad fields of study used in the SDR reporting). In order to create a text classification model, just like we did in the Machine Learning notebooks, you would need to have a training dataset with text and associated field of study label, which you can then use to train your model on and predict the label (field of study) of new, unseen, text.

### Further Resources

Remember to review the Text Analysis chapter in the Big Data and Social Science textbook (edited by Dr. Julia Lane, Dr. Rayid Chani, Dr. Frauke Kreuter) - the link to the textbook  is uploaded on the class website.

Another great resource for NLP in Python is Natural Language Processing with Python: Analyzing Text with the Natural Language Toolkit.

Some of the most popular and useful Python libraries for NLP: 
- NLTK and the associated book/tutorial (www.nltk.org/book), a good introduction to different NLP tasks.
- TextBlob, a simplified library for various NLP tasks (noun phrase extraction, part-of-speech tagging, sentiment analysis, classification, tokenization, etc.).
- spaCy (named entity recognition, part-of-speech tagging, dependency parsing, tokenization, etc.).
- Stanford CoreNLP (while NLTK's emphasis is on simple reference implementations, Stanford CoreNLP is focused no fast implementations of cutting-edge algorithms, particularly for syntactic analysis, e.g. determining the subject of a sentence).